## SHA256 Encryption  

Reference: https://blog.boot.dev/cryptography/how-sha-2-works-step-by-step-sha-256/

### Pre-processing

In [1]:
string="hello world"

The string is converted to ASCII values and then to binary

In [2]:
values=[ord(x) for x in list(string)]
values

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]

In [3]:
values=['{0:08b}'.format(x) for x in values]
values

['01101000',
 '01100101',
 '01101100',
 '01101100',
 '01101111',
 '00100000',
 '01110111',
 '01101111',
 '01110010',
 '01101100',
 '01100100']

In [4]:
inp_length=len(values)*8
inp_length

88

A single 1 bit is added, then the input is padded till its length is divisible by 512  

In [5]:
values.append('10000000')
values

['01101000',
 '01100101',
 '01101100',
 '01101100',
 '01101111',
 '00100000',
 '01110111',
 '01101111',
 '01110010',
 '01101100',
 '01100100',
 '10000000']

The input is padded till the the last 64 digits with zeros

In [6]:
while len(values)*8%512!=0:
    values.append('00000000')
values

['01101000',
 '01100101',
 '01101100',
 '01101100',
 '01101111',
 '00100000',
 '01110111',
 '01101111',
 '01110010',
 '01101100',
 '01100100',
 '10000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000']

The last 64 bits to be appended are formed by the big-endian integer representation of the length of the original binary input.  
In big-endian order, the most significant bits of the sequence are stored first, in the lowest storage address.  

In this example, the length is 88

In [7]:
del values[-9:-1]
values

['01101000',
 '01100101',
 '01101100',
 '01101100',
 '01101111',
 '00100000',
 '01110111',
 '01101111',
 '01110010',
 '01101100',
 '01100100',
 '10000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000']

In [8]:
be_length=list(map('{0:08b}'.format,list((inp_length).to_bytes(8,byteorder="big"))))
be_length

['00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '01011000']

In [9]:
values.extend(be_length)
values

['01101000',
 '01100101',
 '01101100',
 '01101100',
 '01101111',
 '00100000',
 '01110111',
 '01101111',
 '01110010',
 '01101100',
 '01100100',
 '10000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '01011000']

The input sequence has now been created.

### Initializing hash values

In [10]:
from math import sqrt,modf

In [11]:
def isPrime(n):
    c=0
    for i in range(1,n+1):
        if n%i==0:
            c=c+1
    return c==2

8 hash values, which are derived from the first 32 bits of the irrational parts of the square roots of the first 8 prime numbers are created.

In [12]:
H={}
c=0
n=0
while len(H)<8:
    if isPrime(n):
        H[c]=hex(int(modf(sqrt(n))[0]*(1<<32)))
        c+=1
    n+=1
    
H

{0: '0x6a09e667',
 1: '0xbb67ae85',
 2: '0x3c6ef372',
 3: '0xa54ff53a',
 4: '0x510e527f',
 5: '0x9b05688c',
 6: '0x1f83d9ab',
 7: '0x5be0cd19'}

### Initializing round constants

In a similar manner, Round constants are initialized. These are formed from the first 32 bits of the irrational parts of the cube roots of the first 64 prime numbers.

In [13]:
k={}
c=0
n=0
while len(k)<64:
    if isPrime(n):
        k[c]=hex(int(modf(n**(1/3))[0]*(1<<32)))
        c+=1
    n+=1
    
k

{0: '0x428a2f98',
 1: '0x71374491',
 2: '0xb5c0fbcf',
 3: '0xe9b5dba5',
 4: '0x3956c25b',
 5: '0x59f111f1',
 6: '0x923f82a4',
 7: '0xab1c5ed5',
 8: '0xd807aa98',
 9: '0x12835b01',
 10: '0x243185be',
 11: '0x550c7dc3',
 12: '0x72be5d74',
 13: '0x80deb1fe',
 14: '0x9bdc06a7',
 15: '0xc19bf174',
 16: '0xe49b69c1',
 17: '0xefbe4786',
 18: '0xfc19dc6',
 19: '0x240ca1cc',
 20: '0x2de92c6f',
 21: '0x4a7484aa',
 22: '0x5cb0a9dc',
 23: '0x76f988da',
 24: '0x983e5152',
 25: '0xa831c66d',
 26: '0xb00327c8',
 27: '0xbf597fc7',
 28: '0xc6e00bf3',
 29: '0xd5a79147',
 30: '0x6ca6351',
 31: '0x14292967',
 32: '0x27b70a85',
 33: '0x2e1b2138',
 34: '0x4d2c6dfc',
 35: '0x53380d13',
 36: '0x650a7354',
 37: '0x766a0abb',
 38: '0x81c2c92e',
 39: '0x92722c85',
 40: '0xa2bfe8a1',
 41: '0xa81a664b',
 42: '0xc24b8b70',
 43: '0xc76c51a3',
 44: '0xd192e819',
 45: '0xd6990624',
 46: '0xf40e3585',
 47: '0x106aa070',
 48: '0x19a4c116',
 49: '0x1e376c08',
 50: '0x2748774c',
 51: '0x34b0bcb5',
 52: '0x391c0cb3',
 53: 

### Chunk loop  
The following process is applied to every 512-bit portion of the input.  
In this example, the input requires only one such chunk.

#### Creating message schedule

The input is split into 32-bit words and added to a new array.

In [14]:
message_block=[]
message_block.extend(["".join(values)[i:i+32] for i in range(0,512,32)])
message_block

['01101000011001010110110001101100',
 '01101111001000000111011101101111',
 '01110010011011000110010010000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000001011000']

48 words consisting of 32 0 bits are added.  
The array now has 64 32-bit words.

In [15]:
message_block.extend(['0'*32 for _ in range(48)])
message_block

['01101000011001010110110001101100',
 '01101111001000000111011101101111',
 '01110010011011000110010010000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000001011000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 

The indices 16-63 of the array which consist of 0-bits are modified as follows:

For a word w[i] at index i between 16-63:  
* w[i] = w[i-16] + s0 + w[i-7] + s1  


Where  
* s0 = (w[i-15] rightrotate 7) XOR (w[i-15] rightrotate 18) XOR (w[i-15] rightshift 3)
* s1 = (w[i-2] rightrotate 17) XOR (w[i-2] rightrotate 19) XOR (w[i-2] rightshift 10)

It should be noted that all the addition operations are done modulo 2^32

The below function rotates a string (in this case, a string representing a binary number) to the right by the specified number of places:

In [16]:
def rightRotate(string,places):
    return string[len(string)-places:]+string[:len(string)-places]

In [17]:
rightRotate("00110010",2)

'10001100'

The below function achieves the output of bitwise right shift, but ensures that preceeding zeros are added to the string:

In [18]:
def rightShift(string,places):
    return '0'*places+string[:len(string)-places]

In [19]:
rightShift("101010",3)

'000101'

The function below modifies the message block to create the message schedule using the abovementioned functions and algorithm:

In [20]:
def messageSchedule(w):
    for i in range(16,64):
        
        s0=int(rightRotate(w[i-15],7),2)^int(rightRotate(w[i-15],18),2)^int(rightShift(w[i-15],3),2)
        
        s1=int(rightRotate(w[i-2],17),2)^int(rightRotate(w[i-2],19),2)^int(rightShift(w[i-2],10),2)
        
        new_wi=int(w[i-16],2)+s0+int(w[i-7],2)+s1
        new_wi=new_wi%(2**32)
        new_wi='{0:08b}'.format(new_wi)
        new_wi='0'*(32-len(new_wi))+new_wi
        
        w[i]=new_wi

In [21]:
messageSchedule(message_block)

In [22]:
message_block

['01101000011001010110110001101100',
 '01101111001000000111011101101111',
 '01110010011011000110010010000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000001011000',
 '00110111010001110000001000110111',
 '10000110110100001100000000110001',
 '11010011101111010001000100001011',
 '01111000001111110100011110000010',
 '00101010100100000111110011101101',
 '01001011001011110111110011001001',
 '00110001111000011001010001011101',
 '10001001001101100100100101100100',
 '01111111011110100000011011011010',
 '11000001011110011010100100111010',
 '10111011111010001111011001010101',
 

#### Compression

8 variables a,b,c,d,e,f,g, and h are initialized and set as the current hash values.

In [23]:
H={x:int(y,16) for x,y in list(H.items())}
H

{0: 1779033703,
 1: 3144134277,
 2: 1013904242,
 3: 2773480762,
 4: 1359893119,
 5: 2600822924,
 6: 528734635,
 7: 1541459225}

In [24]:
H={x:'{0:08b}'.format(y) for x,y in list(H.items())}
H

{0: '1101010000010011110011001100111',
 1: '10111011011001111010111010000101',
 2: '111100011011101111001101110010',
 3: '10100101010011111111010100111010',
 4: '1010001000011100101001001111111',
 5: '10011011000001010110100010001100',
 6: '11111100000111101100110101011',
 7: '1011011111000001100110100011001'}

In [25]:
H={x:"0"*(32-len(y))+y for x,y in list(H.items())}
H

{0: '01101010000010011110011001100111',
 1: '10111011011001111010111010000101',
 2: '00111100011011101111001101110010',
 3: '10100101010011111111010100111010',
 4: '01010001000011100101001001111111',
 5: '10011011000001010110100010001100',
 6: '00011111100000111101100110101011',
 7: '01011011111000001100110100011001'}

A compression loop is run, where the values of the variables a-h are mutated as follows: 

For index i from 0 to 63:  
* h = g
* g = f
* f = e
* e = d + temp1
* d = c
* c = b
* b = a
* a = temp1 + temp2  

Where:
* temp1 = h + s1 + ch + k[i] + w[i]
* temp2 = s0 + maj  

* s1 = (e rightrotate 6) XOR (e rightrotate 11) XOR (e rightrotate 25)
* ch = (e AND f) XOR ((NOT e) AND g)
* s0 = (a rightrotate 2) XOR (a rightrotate 13) XOR (a rightrotate 22)
* maj = (a AND b) XOR (a AND c) XOR (b AND c)

In [26]:
def compression(w,H):
    a,b,c,d,e,f,g,h=list(H.values())
    
    for i in range(64):
        
        s1 = int(rightRotate(e,6),2)^int(rightRotate(e,11),2)^int(rightRotate(e,25),2)
        
        ch = (int(e,2)&int(f,2))^((~(int(e,2)))&int(g,2))
        
        temp1 = int(h,2) + s1 + ch + int(k[i],16) + int(w[i],2)
        temp1 = temp1%(2**32)
        
        s0 = int(rightRotate(a,2),2)^int(rightRotate(a,13),2)^int(rightRotate(a,22),2)
        
        maj = (int(a,2)&int(b,2))^(int(a,2)&int(c,2))^(int(b,2)&int(c,2))
        
        temp2 = s0+maj
        temp2 = temp2%(2**32)
        
        h=g
        
        g=f
        
        f=e
        
        e=int(d,2)+temp1
        e=e%(2**32)
        e='{0:08b}'.format(e)
        
        d=c
        
        c=b
        
        b=a
        
        a=temp1+temp2
        a=a%(2**32)
        a='{0:08b}'.format(a)
        
        a="0"*(32-len(a))+a
        b="0"*(32-len(b))+b
        c="0"*(32-len(c))+c
        d="0"*(32-len(d))+d
        e="0"*(32-len(e))+e
        f="0"*(32-len(f))+f
        g="0"*(32-len(g))+g
        h="0"*(32-len(h))+h
        
    return a,b,c,d,e,f,g,h
        

In [27]:
var=compression(message_block,H)

In [28]:
var

('01001111010000110100000101010010',
 '11010111111001011000111110000011',
 '01101000101111110101111101100101',
 '00110101001011011011011011000000',
 '01110011011101101001110101100100',
 '11011111010011100001100001100010',
 '01110001000001010001111000000001',
 '10000111000011110000000011010000')

#### Modifying final values

After compression, within the chunk loop, the hash values are modified by adding their respective variables to them, as follows:  

h0 = h0 + a  
h1 = h1 + b  
h2 = h2 + c  
     .  
     .  
     .  
h7 = h7 + h     

In [29]:
for i in range(8):
    H[i]=int(H[i],2)+int(var[i],2)
    H[i]=H[i]%(2**32)
    H[i]='{0:08b}'.format(H[i])

In [30]:
H

{0: '10111001010011010010011110111001',
 1: '10010011010011010011111000001000',
 2: '10100101001011100101001011010111',
 3: '11011010011111011010101111111010',
 4: '11000100100001001110111111100011',
 5: '1111010010100111000000011101110',
 6: '10010000100010001111011110101100',
 7: '11100010111011111100110111101001'}

#### Concatenation of final hash

Concatenate the modified hashes to obtain final hash

In [31]:
calculated_sha256="".join(["{:x}".format(int(x,2)) for x in H.values()])
calculated_sha256

'b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9'

In [32]:
import hashlib

In [33]:
real_sha256=hashlib.sha256("hello world".encode('utf-8')).hexdigest()
real_sha256

'b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9'

In [34]:
calculated_sha256 == real_sha256

True